<a href="https://colab.research.google.com/github/pkant-0/Recommendation-Systems/blob/main/Content_Based_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Scenario**

A pharma company wants to recommend medicines to pharmacies based on their purchase history. The goal is to recommend medicines similar to the ones already purchased.

## **Step 1: Define the problem**

### Input: Dataset containing details of medicines and purchase history.

### Output: Recommendations for medicines that are similar to those purchased.

### Approach: Use Content-Based Filtering to recommend medicines based on their attributes.

## **Step 2: Dataset Preparation**
Synthetic Dataset

    Medicine Attributes Dataset:
        Medicine ID, Name, Ingredients, Category, Manufacturer, Price.

Medicine_ID	Name	Ingredients	Category	Manufacturer	Price
101	Paracetamol	Paracetamol, Caffeine	Pain Relief	PharmaCorp	50.00
102	Ibuprofen	Ibuprofen	Pain Relief	MedicoLtd	75.00
103	Cetirizine	Cetirizine	Allergy	PharmaCorp	40.00
104	Amoxicillin	Amoxicillin	Antibiotic	MedicoLtd	90.00

##    Purchase History Dataset:
        Pharmacy ID, Medicine IDs purchased.

Pharmacy_ID	Purchased_Medicines
P1	[101, 102]
P2	[103, 104]


Step 3: Preprocessing
Actions:

    Tokenize Text Features:
        Convert text features like Ingredients and Category into numerical formats.

    Create Medicine Profiles:
        Combine all attributes of each medicine into a single text field.

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Medicine Attributes

medicine_data = pd.DataFrame({
    'Medicine_ID': [101, 102, 103, 104],
    'Name': ['Paracetamol', 'Ibuprofen', 'Cetirizine', 'Amoxicillin'],
    'Ingredients': ['Paracetamol, Caffeine', 'Ibuprofen', 'Cetirizine', 'Amoxicillin'],
    'Category': ['Pain Relief', 'Pain Relief', 'Allergy', 'Antibiotic'],
    'Manufacturer': ['PharmaCorp', 'MedicoLtd', 'PharmaCorp', 'MedicoLtd'],
    'Price': [50.00, 75.00, 40.00, 90.00]
})

# Combine Features into a single Field
medicine_data['Profile'] = (
    medicine_data['Ingredients'] + " " +
    medicine_data['Category'] + " " +
    medicine_data['Manufacturer'])


In [5]:
# View profiles
print(medicine_data[['Medicine_ID', 'Profile']])

   Medicine_ID                                       Profile
0          101  Paracetamol, Caffeine Pain Relief PharmaCorp
1          102               Ibuprofen Pain Relief MedicoLtd
2          103                 Cetirizine Allergy PharmaCorp
3          104              Amoxicillin Antibiotic MedicoLtd


# Step 4: Compute Similarities
TF-IDF Vectorization:

Convert the Profile field into numerical vectors using TF-IDF.

In [7]:
# Vectorize Medicine Profiles

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(medicine_data['Profile'])

# Compute Cosine Similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Convert to DataFrame

cosine_sim_df = pd.DataFrame(cosine_sim, index=medicine_data['Medicine_ID'], columns=medicine_data['Medicine_ID'])

# **Recomendation Fuction**

## Logic:
1. Find medicines most similar to the ones purchased by a pharmacy.
2. Rank them by similarity scores.

In [17]:
def recommend_medicines(pharmacy_id, purchase_history, medicine_data, similarity_matrix, top_n=3):
# Get purchased medicines
  purchased = purchase_history.loc[purchase_history['Pharmacy_ID'] == pharmacy_id, 'Purchased_Medicines'].values[0]

# Calculate similarity scores for all medicines

  scores = similarity_matrix.loc[purchased].sum(axis=0)

# Exclude alredy purchased medicines
  scores = scores.drop(index=purchased)

# Get top N recommendations
  recommendations = scores.sort_values(ascending=False).head(top_n).index
  return medicine_data[medicine_data['Medicine_ID'].isin(recommendations)]

# Purchase History
purchase_history = pd.DataFrame({
    'Pharmacy_ID': ['P1', 'P2'],
    'Purchased_Medicines': [[101, 102], [103, 104]]
})

# Recommend for Pharmacy P1
recommendations = recommend_medicines('P1', purchase_history, medicine_data, cosine_sim_df)
print(recommendations[['Medicine_ID', 'Name']])

   Medicine_ID         Name
2          103   Cetirizine
3          104  Amoxicillin


# **Model Integration**

In [18]:
from flask import Flask, request, jsonify
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

app = Flask(__name__)

# Sample Medicine Dataset
medicine_data = pd.DataFrame({
    'Medicine_ID': [101, 102, 103, 104],
    'Name': ['Paracetamol', 'Ibuprofen', 'Cetirizine', 'Amoxicillin'],
    'Ingredients': ['Paracetamol, Caffeine', 'Ibuprofen', 'Cetirizine', 'Amoxicillin'],
    'Category': ['Pain Relief', 'Pain Relief', 'Allergy', 'Antibiotic'],
    'Manufacturer': ['PharmaCorp', 'MedicoLtd', 'PharmaCorp', 'MedicoLtd'],
    'Price': [50.00, 75.00, 40.00, 90.00]
})

# Create a Combined Profile
medicine_data['Profile'] = (
    medicine_data['Ingredients'] + " " +
    medicine_data['Category'] + " " +
    medicine_data['Manufacturer']
)

# Vectorize Profiles
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(medicine_data['Profile'])

# Compute Similarity Matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim_df = pd.DataFrame(cosine_sim, index=medicine_data['Medicine_ID'], columns=medicine_data['Medicine_ID'])

# Recommendation Function
def recommend_medicines(purchased, top_n=3):
    scores = cosine_sim_df.loc[purchased].sum(axis=0)
    scores = scores.drop(index=purchased)  # Exclude already purchased medicines
    recommendations = scores.sort_values(ascending=False).head(top_n).index
    return medicine_data[medicine_data['Medicine_ID'].isin(recommendations)][['Medicine_ID', 'Name']].to_dict(orient='records')

# API Route
@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.get_json()
    pharmacy_id = data.get('pharmacy_id')
    purchased = data.get('purchased')
    recommendations = recommend_medicines(purchased)
    return jsonify({'pharmacy_id': pharmacy_id, 'recommendations': recommendations})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
